In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import AlgoLoop
import StatisticFunc

plt.style.use('seaborn-deep')                     # 绘图风格
matplotlib.rcParams['font.sans-serif'] = ['SimHei']  # 字体雅黑
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False    # 处理负号

# 预处理

## 提取并处理数据

### 各种利率

In [ ]:
# GC007 利率
GCRates = pd.read_excel(io="Raw/GC007利率.xlsx")              # 读取数据
GCRates = GCRates.drop([0, 1, 2], axis=0)                    # 删除多余行
GCRates.columns = ['日期', 'GC007']                           # 修改列名
GCRates["日期"] = pd.DatetimeIndex(GCRates["日期"])           # 类型转换
GCRates = GCRates.set_index("日期")                          # 设置索引

In [ ]:
# 10年国债收益率
CNRates = pd.read_excel(io="Raw/10年国债到期收益率.xls")        # 读取数据
CNRates.columns = ['日期', '十年国债收益率']                             # 修改列名
CNRates = CNRates.drop([0, 3870, 3871], axis=0)                       # 删除多余行
CNRates["日期"] = pd.DatetimeIndex(CNRates["日期"])                    # 类型转换
CNRates = CNRates.set_index("日期")                                   # 设置索引

In [ ]:
# 10年美债收益率
USRates = pd.read_excel(io="Raw/10年美债到期收益率.xlsx")                       # 读取数据
USRates.columns = ['日期', '十年美债收益率']                             # 修改列名
USRates = USRates.drop([0, 6263, 6264], axis=0)                       # 删除多余行
USRates["日期"] = pd.DatetimeIndex(USRates["日期"])                    # 类型转换
USRates = USRates.set_index("日期")                                    # 设置索引

In [ ]:
# 合并利率
Rates = GCRates.merge(CNRates, how='left', left_index=True, right_index=True)
Rates = Rates.merge(USRates, how='left', left_index=True, right_index=True)

### 权益类资产换手率

In [ ]:
Turnovers = pd.read_excel(io="Raw/资产换手率.xlsx")                         # 读取数据
Turnovers = Turnovers.drop([0], axis=0)                                   # 删除多余行
Turnovers["日期"] = pd.DatetimeIndex(Turnovers["日期"])                    # 类型转换
Turnovers = Turnovers.set_index("日期")                                   # 设置索引

In [ ]:
# 时间对齐
Turnovers = Turnovers.loc[Rates.index, :]

### 铁矿石价格指数

In [ ]:
IronStone = pd.read_excel(io="Raw/铁矿石价格指数.xls")                       # 读取数据
IronStone = IronStone.drop([0, 2642, 2643], axis=0)                       # 删除多余行
IronStone.columns=['日期', '铁矿石期货']                                     # 修改列名
IronStone["日期"] = pd.DatetimeIndex(IronStone["日期"])                    # 类型转换
IronStone = IronStone.set_index("日期")                                   # 设置索引

### 各资产收盘价

In [ ]:
Assets = pd.read_excel(io="Raw/资产收盘价.xlsx")                      # 读取数据
Assets = Assets.drop([0], axis=0)                                   # 删除多余行
Assets["日期"] = pd.DatetimeIndex(Assets["日期"])                    # 类型转换
Assets = Assets.set_index("日期")                                   # 设置索引
Assets = Assets.loc[Rates.index, :]                                # 时间对齐

In [ ]:
# 整合铁矿石数据
Assets = Assets.drop('中信证券铁矿石', axis=1)
Assets = Assets.merge(IronStone, how='left', left_index=True, right_index=True)

# 填充铁矿石2011年前缺失数据
Assets['铁矿石期货'] = Assets['铁矿石期货'].fillna(method='bfill')
Assets.head()

### 十年国债&美债价格指数

In [ ]:
HoldPeriod = (Assets.index - Assets.index[0]).days

# 计算十年国债
Assets['10年国债'] = 100 - (Rates['十年国债收益率'] - 3.0) * 8.2 + 3.0 * HoldPeriod / 365.0
Assets['10年国债'] = Assets['10年国债'].fillna(method='ffill')

#计算十年美债
Assets['10年美债'] = 100 - (Rates['十年美债收益率'] - 3.0) * 8.2 + 3.0 * HoldPeriod / 365.0
Assets['10年美债'] = Assets['10年美债'].fillna(method='ffill')

In [ ]:
# 替换上证十年国债指数
Assets = Assets.drop('上证10年国债', axis=1)
Assets.head()

### 美元汇率

In [ ]:
FXRates = pd.read_excel(io="Raw/美元汇率.xlsx")
FXRates = FXRates.drop([0, 1, 2], axis=0)
FXRates.columns = ['日期', '美元汇率']                        
FXRates["日期"] = pd.DatetimeIndex(FXRates["日期"])
FXRates = FXRates.set_index("日期")

In [ ]:
# # 皮尔逊相关系数
# np.corrcoef( FXRates.values.flatten().astype(np.float32)[900:-233], 
#              (Assets['10年国债'] - Assets['10年美债']).values.astype(np.float32)[900:-220] )

## 计算各资产收益率

### 日内损益

In [ ]:
Returns = Assets.pct_change(axis=0)
Returns = Returns.dropna(axis=0, how='all')           # 删除无数据日

### 累计损益

In [ ]:
cumReturns = (1.0 + Returns)
cumReturns = cumReturns.fillna(1.0)                    # 填充空值
cumReturns = cumReturns.cumprod()                      # 计算各资产累计收益率

### 杠杆调整后的债券净值

In [ ]:
lever = 2.0                                 # 杠杆（额外）
leverReturns = Returns.copy()               # 创建副本 
leverCumReturns = cumReturns.copy()   

In [ ]:
# 更新杠杆调整后的债券净值
leverReturns['10年国债'] = Returns['10年国债'] * (1.0 + lever) - Rates.loc[Returns.index, 'GC007'] * lever / (365.0 * 100)
leverReturns['信用债3-5AAA'] = Returns['信用债3-5AAA'] * (1.0 + lever) - Rates.loc[Returns.index, 'GC007'] * lever / (365.0 * 100)

# 更新累计净值
leverCumReturns = (1.0 + leverReturns)
leverCumReturns = leverCumReturns.fillna(1.0)                    # 填充空值
leverCumReturns = leverCumReturns.cumprod()                      # 计算各资产累计收益率

### 剔除美元资产

In [ ]:
leverReturns_Domestic    = leverReturns.drop(['标普500', '10年美债'], axis=1)
leverCumReturns_Domestic = leverCumReturns.drop(['标普500', '10年美债'], axis=1)

In [ ]:
leverCumReturns_Domestic.plot(figsize=(16, 6))

# 风险平价 （杠杆+指数平均）

## 程序运行

In [ ]:
tradeDF, weightDF = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                       Turnovers, FXRates, mode='ema')

In [ ]:
StatisticFunc.WeightPlot(tradeDF, weightDF, '基准模型')

## 回测表现汇总

In [ ]:
smryDF = StatisticFunc.summaryDF(tradeDF)
pfmcDF = StatisticFunc.performanceDF(smryDF, tradeDF, name='基准模型')

In [ ]:
smryDF

In [ ]:
pfmcDF

## 收益率贡献度

In [ ]:
annualContrb = StatisticFunc.AnnualContribution(tradeDF)

In [ ]:
StatisticFunc.BarPlot(annualContrb, '基准模型')

## 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 6))

plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, label='基准模型')
    
plt.plot(cumReturns.index, cumReturns['中证500'], lw=0.8, label='中证500')
plt.plot(cumReturns.index, cumReturns['标普500'], lw=0.8, label='标普500')
plt.plot(cumReturns.index, cumReturns['10年国债'], lw=0.8, label='10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（杠杆，指数平均）')
plt.savefig('Pics/净值曲线（杠杆，指数平均）.png')

# 风险平价 + 单因子策略（杠杆+指数平均）

## 换手率

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':True, 'fxRate':False,
            'copperGold':False, 'copperGas':False}

tradeDF_1, weightDF_1 = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                           Turnovers, FXRates, mode='ema', dt=120, up=0.50, 
                                           thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_1, weightDF_1, '换手率')

### 回测表现汇总

In [ ]:
smryDF_1 = StatisticFunc.summaryDF(tradeDF_1)
pfmcDF_1 = StatisticFunc.performanceDF(smryDF_1, tradeDF_1, name='换手率')

In [ ]:
smryDF_1

In [ ]:
pfmcDF_1

### 收益率贡献度

In [ ]:
annualContrb_1 = StatisticFunc.AnnualContribution(tradeDF_1)

In [ ]:
StatisticFunc.BarPlot(annualContrb_1, '换手率')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_1.index, tradeDF_1['投资组合净值']/10000, label='换手率')

# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='基准组合')     
plt.plot(cumReturns.index, cumReturns['中证500'], label='中证500')
plt.plot(cumReturns.index, cumReturns['10年国债'], label='中国10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 5.0)
plt.legend(loc='upper left')
plt.title('净值曲线（换手率）')
plt.savefig('Pics/净值曲线（换手率）.png')

## 横截面动量

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':True, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 'fxRate':False,
            'copperGold':False, 'copperGas':False}

tradeDF_2, weightDF_2 = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                           Turnovers, FXRates, mode='ema', dt=120, up=0.50, 
                                           thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_2, weightDF_2, '横截面动量')

### 回测表现汇总

In [ ]:
smryDF_2 = StatisticFunc.summaryDF(tradeDF_2)
pfmcDF_2 = StatisticFunc.performanceDF(smryDF_2, tradeDF_2, name='横截面动量')

In [ ]:
smryDF_2

In [ ]:
pfmcDF_2

### 收益率贡献度

In [ ]:
annualContrb_2 = StatisticFunc.AnnualContribution(tradeDF_2)

In [ ]:
StatisticFunc.BarPlot(annualContrb_2, '横截面动量')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_2.index, tradeDF_2['投资组合净值']/10000, label='横截面动量')
        
# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='基准组合')
plt.plot(cumReturns.index, cumReturns['中证500'], lw=0.8, label='中证500')
plt.plot(cumReturns.index, cumReturns['标普500'], lw=0.8, label='标普500')
plt.plot(cumReturns.index, cumReturns['10年国债'], lw=0.8, label='10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（横截面动量）')
plt.savefig('Pics/净值曲线（横截面动量）.png')

## 时序动量

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':True, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 'fxRate':False,
            'copperGold':False, 'copperGas':False}

tradeDF_3, weightDF_3 = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                           Turnovers, FXRates, mode='ema', dt=120, up=0.50, 
                                           thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_3, weightDF_3, '时序动量')

### 回测表现汇总

In [ ]:
smryDF_3 = StatisticFunc.summaryDF(tradeDF_3)
pfmcDF_3 = StatisticFunc.performanceDF(smryDF_3, tradeDF_3, name='时序动量')

In [ ]:
smryDF_3

In [ ]:
pfmcDF_3

### 收益率贡献度

In [ ]:
annualContrb_3 = StatisticFunc.AnnualContribution(tradeDF_3)

In [ ]:
StatisticFunc.BarPlot(annualContrb_3, '时序动量')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_3.index, tradeDF_3['投资组合净值']/10000, label='时序动量')
    
# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='基准组合') 
plt.plot(cumReturns.index, cumReturns['中证500'], lw=0.8, label='中证500')
plt.plot(cumReturns.index, cumReturns['标普500'], lw=0.8, label='标普500')
plt.plot(cumReturns.index, cumReturns['10年国债'], lw=0.8, label='10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（时序动量）')
plt.savefig('Pics/净值曲线（时序动量）D.png')

## 汇率因子

### 程序运行

In [ ]:
# thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
# factors = {'momentumX':False, 'momentumT':False, 
#             'reverseX':False, 'reverseT':False,
#             'turnover':False, 'fxRate':True,
#             'copperGold':True, 'copperGas':False}

# tradeDF_4, weightDF_4 = AlgoLoop.AlgoTrade(Assets, leverReturns, leverCumReturns, 
#                                            Turnovers, FXRates, mode='ema', dt=120, up=0.50, 
#                                            thresholds=thrds, factorDict=factors)

In [ ]:
# StatisticFunc.WeightPlot(tradeDF_4, weightDF_4, '汇率因子')

### 回测表现汇总

In [ ]:
# smryDF_4 = StatisticFunc.summaryDF(tradeDF_4)
# pfmcDF_4 = StatisticFunc.performanceDF(smryDF_4, tradeDF_4, name='汇率因子')

In [ ]:
# smryDF_4

In [ ]:
# pfmcDF_4

### 收益率贡献度

In [ ]:
# annualContrb_4 = StatisticFunc.AnnualContribution(tradeDF_4)

In [ ]:
# StatisticFunc.BarPlot(annualContrb_4, '汇率因子')

### 投资组合净值曲线

In [ ]:
# fig = plt.figure(figsize=(16, 8))

# plt.plot(tradeDF_4.index, tradeDF_4['投资组合净值']/10000, label='汇率因子')
    
    
# # 基准参考
# plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='基准组合') 
# plt.plot(cumReturns.index, cumReturns['中证500'], lw=0.8, label='中证500')
# plt.plot(cumReturns.index, cumReturns['标普500'], lw=0.8, label='标普500')
# plt.plot(cumReturns.index, cumReturns['10年国债'], lw=0.8, label='10年国债')

# plt.xlabel('时间')
# plt.ylabel('净值')
# plt.ylim(0.0, 3.0)
# plt.legend(loc='upper left')
# plt.title('净值曲线（汇率因子）')
# plt.savefig('Pics/净值曲线（汇率因子）.png')

## 铜金

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 'fxRate':False,
            'copperGold':True, 'copperGas':False}

tradeDF_5, weightDF_5 = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                           Turnovers, FXRates, mode='ema', dt=120, up=0.50, thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_5, weightDF_5, '铜金比')

### 回测表现汇总

In [ ]:
smryDF_5 = StatisticFunc.summaryDF(tradeDF_5)
pfmcDF_5 = StatisticFunc.performanceDF(smryDF_5, tradeDF_5, name='铜金比')

In [ ]:
smryDF_5

In [ ]:
pfmcDF_5

### 收益率贡献度

In [ ]:
annualContrb_5 = StatisticFunc.AnnualContribution(tradeDF_5)

In [ ]:
StatisticFunc.BarPlot(annualContrb_5, '铜金比')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_5.index, tradeDF_5['投资组合净值']/10000, label='铜金比')
    
# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='基准组合')   
plt.plot(cumReturns.index, cumReturns['中证500'], lw=0.8, label='中证500')
plt.plot(cumReturns.index, cumReturns['标普500'], lw=0.8, label='标普500')
plt.plot(cumReturns.index, cumReturns['10年国债'], lw=0.8, label='10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（铜金比）')
plt.savefig('Pics/净值曲线（铜金比）.png')

## 铜油

### 程序运行

In [ ]:
thrds = {'Equity':0.50, 'FixedIncome':0.90, 'Commodity':0.30}
factors = {'momentumX':False, 'momentumT':False, 
            'reverseX':False, 'reverseT':False,
            'turnover':False, 'fxRate':False,
            'copperGold':False, 'copperGas':True}

tradeDF_6, weightDF_6 = AlgoLoop.AlgoTrade(Assets, leverReturns_Domestic, leverCumReturns_Domestic, 
                                           Turnovers, FXRates, mode='ema', dt=120, up=0.50, 
                                           thresholds=thrds, factorDict=factors)

In [ ]:
StatisticFunc.WeightPlot(tradeDF_6, weightDF_6, '铜油比')

### 回测表现汇总

In [ ]:
smryDF_6 = StatisticFunc.summaryDF(tradeDF_6)
pfmcDF_6 = StatisticFunc.performanceDF(smryDF_6, tradeDF_6, name='铜油比')

In [ ]:
smryDF_6

In [ ]:
pfmcDF_6

### 收益率贡献度

In [ ]:
annualContrb_6 = StatisticFunc.AnnualContribution(tradeDF_6)

In [ ]:
StatisticFunc.BarPlot(annualContrb_6, '铜油比')

### 投资组合净值曲线

In [ ]:
fig = plt.figure(figsize=(16, 8))

plt.plot(tradeDF_6.index, tradeDF_6['投资组合净值']/10000, label='铜油比')

# 基准参考
plt.plot(tradeDF.index, tradeDF['投资组合净值']/10000, ls='--', label='基准组合') 
plt.plot(cumReturns.index, cumReturns['中证500'], lw=0.8, label='中证500')
plt.plot(cumReturns.index, cumReturns['标普500'], lw=0.8, label='标普500')
plt.plot(cumReturns.index, cumReturns['10年国债'], lw=0.8, label='10年国债')

plt.xlabel('时间')
plt.ylabel('净值')
plt.ylim(0.0, 3.0)
plt.legend(loc='upper left')
plt.title('净值曲线（铜油比）')
plt.savefig('Pics/净值曲线（铜油比).png')

## 汇总比较

In [ ]:
pd.concat([pfmcDF, pfmcDF_1, pfmcDF_2, pfmcDF_3, pfmcDF_5, pfmcDF_6])